# Creating Custom Rheological Models

Learn to create custom models by inheriting from BaseModel with automatic Bayesian capabilities.

## Learning Objectives
- Inherit from BaseModel to create custom rheological model
- Implement _fit() and _predict() methods
- Register with ModelRegistry for ecosystem integration
- Demonstrate automatic Bayesian inference via BayesianMixin
- Test and validate custom model

## Prerequisites
- Model fitting basics (Phase 1)
- Understanding of Zener and Maxwell models

**Estimated Time:** 50-55 minutes

## Example: Burgers Model (Zener + Maxwell in Series)

Implement 4-parameter Burgers model demonstrating BaseModel inheritance pattern.

In [ ]:
from rheo.core.base import BaseModel
from rheo.core.parameters import Parameter, ParameterSet
from rheo.core.registry import ModelRegistry
from rheo.core.jax_config import safe_import_jax
import numpy as np

jax, jnp = safe_import_jax()

print('✓ Imports for custom model development')

## Burgers Model Implementation

In [ ]:
@ModelRegistry.register('burgers')
class BurgersModel(BaseModel):
    '''Burgers model: Zener + Maxwell in series.
    
    4 parameters: Ge, Gm1, eta1, eta2
    G(t) = Ge + Gm1*exp(-t*Gm1/eta1) + (Gm1*eta2/eta1)*t
    '''
    
    def __init__(self):
        super().__init__()
        self.parameters = ParameterSet([
            Parameter('Ge', value=1e4, bounds=(1e2, 1e6), units='Pa'),
            Parameter('Gm1', value=5e4, bounds=(1e3, 1e7), units='Pa'),
            Parameter('eta1', value=1e3, bounds=(1e1, 1e5), units='Pa·s'),
            Parameter('eta2', value=1e4, bounds=(1e2, 1e6), units='Pa·s')
        ])
    
    def _predict(self, X, test_mode=None):
        '''Predict Burgers response.'''
        Ge = self.parameters.get_value('Ge')
        Gm1 = self.parameters.get_value('Gm1')
        eta1 = self.parameters.get_value('eta1')
        eta2 = self.parameters.get_value('eta2')
        
        # Burgers relaxation
        tau1 = eta1 / Gm1
        return Ge + Gm1 * jnp.exp(-X / tau1) + (Gm1 * eta2 / eta1) * X

print('✓ Burgers model implemented and registered')

## Test Custom Model

Validate that custom model fits data correctly.

In [ ]:
# Generate Burgers data
t = np.logspace(-2, 2, 50)
Ge, Gm1, eta1, eta2 = 1e4, 5e4, 1e3, 1e4
G_t_true = Ge + Gm1 * np.exp(-t * Gm1 / eta1) + (Gm1 * eta2 / eta1) * t
G_t_noisy = G_t_true + np.random.normal(0, 0.02 * G_t_true)

# Fit custom model
model = BurgersModel()
model.fit(t, G_t_noisy)

print('Fitted Burgers Parameters:')
for param in ['Ge', 'Gm1', 'eta1', 'eta2']:
    print(f'  {param}: {model.parameters.get_value(param):.2e}')

## Automatic Bayesian Inference

Custom models automatically inherit Bayesian capabilities.

In [ ]:
# Bayesian inference works automatically!
result = model.fit_bayesian(
    t, G_t_noisy,
    num_warmup=500,
    num_samples=1000,
    num_chains=1
)

print('\nBayesian inference on custom model: ✓')
print('Convergence:')
for param, r_hat in result.diagnostics['r_hat'].items():
    print(f'  {param}: R-hat = {r_hat:.4f}')

## Key Takeaways

- **BaseModel Inheritance:** Provides full Rheo ecosystem integration
- **Automatic Bayesian:** BayesianMixin gives free NUTS inference
- **ModelRegistry:** Makes model available throughout Rheo
- **Testing:** Always validate with synthetic data

## Next Steps
- **[04-fractional-models-deep-dive.ipynb](04-fractional-models-deep-dive.ipynb):** Advanced fractional models
- **[../basic/01-maxwell-fitting.ipynb](../basic/01-maxwell-fitting.ipynb):** Review BaseModel usage